In [1]:
from random import randrange
import numpy as np
import cv2
import os
import glob

In [2]:
def get_filename(fulllocation):
    i=1
    while True:
        if fulllocation[-i] is not "/":
            i += 1
        else:
            name=fulllocation[-(i-1):]
            break
    return name

def explore_dir(dir,count):
    if count==0:
        global n_dir, n_file, filenames, filelocations
        n_dir=n_file=0
        filenames=filelocations=np.array([])
    for obj in glob.glob(dir+"/*"):
        if os.path.isdir(obj):
            n_dir +=1
            explore_dir(obj,count+1)
        elif os.path.isfile(obj):
            n_file += 1
            loc=np.array([obj])
            name=np.array([get_filename(obj)])
            filelocations=np.concatenate((filelocations, loc), axis=0)
            filenames=np.concatenate((filenames, name), axis=0)
    return np.array([filenames,filelocations])

In [3]:
def detect(img):
    x_axis, y_axis=img.shape[:2]
    coors=[0,0,0,0,0] # x1,x2,y1,y2,exist
    
    threshold=0
    
    for i in range(x_axis) :
        for j in range(y_axis) :
            if img[i][j][0] > threshold or img[i][j][1] > threshold or img[i][j][2] > threshold :    
                coors[4] =1 # exist
                if coors[0] == 0 and coors[2] ==0:
                    coors[0] = coors[1] = i
                    coors[2] = coors[3] = j
                    
                elif coors[0]<i and coors[1] < i: 
                    coors[1] = i
                elif coors[1]>i :
                    coors[0] = i
                    
                elif coors[2]<j and coors[3] < j: 
                    coors[3] = j
                elif coors[3]>j :
                    coors[2] = j
    return coors

In [4]:
def Resizing_img(image,mask_img,image_name,mask_name):

    coors=detect(mask_img)
    
    cropI = image[coors[0]:(coors[1]+1), coors[2]:(coors[3]+1)]
    resizedI=cv2.resize(cropI, (64, 64))
    cropM = mask_img[coors[0]:(coors[1]+1), coors[2]:(coors[3]+1)]
    resizedM=cv2.resize(cropM, (64, 64))

In [5]:
def Rotate_img(img,i):
    global imgR
    x, y =img.shape[:2]
    rotation_matrix=cv2.getRotationMatrix2D((x/2,y/2),90*(i),1)
    imgR=cv2.warpAffine(img, rotation_matrix, (x,y))
def Mirror_img(img):
    global imgM
    x, y =img.shape[:2]
    src_point = np.float32([[0,0], [y-1,0], [0,x-1]])
    dst_point = np.float32([[y-1,0], [0,0], [y-1, x-1]])
    affine_matrix=cv2.getAffineTransform(src_point, dst_point)
    imgM = cv2.warpAffine(img, affine_matrix, (y,x))

In [6]:
def Crop_ROI(image,mask_img,image_name,mask_name):
    
    coors=detect(mask_img)
    
    if coors[4] == 1:                
        x_size = coors[1] - coors[0] +1
        y_size = coors[3] - coors[2] +1

        if (x_size) !=64 :
            if x_size%2 == 0:
                coors[0]-=(64-x_size)/2
                coors[1]+=(64-x_size)/2
            elif x_size%2 != 0:
                coors[0]-=((64-x_size)/2) 
                coors[1]+=((64-x_size)/2) +1          

        if (y_size) !=64 :
            if y_size%2 == 0:
                coors[2]-=(64-y_size)/2
                coors[3]+=(64-y_size)/2
            elif y_size%2 !=0 :
                coors[2]-=((64-y_size)/2) 
                coors[3]+=((64-y_size)/2) +1 

        cropI = image[int(coors[0]):int(coors[1]+1), int(coors[2]):int(coors[3]+1)]
        cropM = mask_img[int(coors[1]):int(coors[1]+1), int(coors[2]):int(coors[3]+1)]

        for n in range(4):
            Rotate_img(cropI,n)
            cv2.imwrite("./Augmentation/images/"+image_name+"_"+str(n+1)+".jpg", imgR)
            Mirror_img(imgR)
            cv2.imwrite("./Augmentation/images/"+image_name+"_"+str(n+5)+".jpg", imgM)

            Rotate_img(cropM,n)
            cv2.imwrite("./Augmentation/labels/"+mask_name+"_"+str(n+1)+".jpg", imgR)
            Mirror_img(imgR)
            cv2.imwrite("./Augmentation/labels/"+mask_name+"_"+str(n+5)+".jpg", imgM)

In [7]:
def Crop_None(image,mask_img,image_name,mask_name,n):
    Check_overwrapping=1
    coors_mask=detect(mask_img)

    while(1):
        x_coor=randrange(50,462)
        y_coor=randrange(50,462)

        coors_img=[x_coor,x_coor+64,y_coor,y_coor+64]
        
        for s in range(64):
            if coors_mask[0]<(coors_img[0]+s) and (coors_img[0]+s)<coors_mask[1]:
                break
            elif coors_mask[0]<(coors_img[1]+s) and (coors_img[1]+s)<coors_mask[1]:
                break
            elif coors_mask[2]<(coors_img[2]+s) and (coors_img[2]+s)<coors_mask[3]:
                break
            elif coors_mask[2]<(coors_img[3]+s) and (coors_img[3]+s)<coors_mask[3]:
                break
            elif (coors_img[3]+s)>462 or (coors_img[1]+s) >462:
                break
            elif (coors_img[3]+s)<50 or (coors_img[1]+s) <50:
                break
            else:
                Check_overwrapping=0
        if Check_overwrapping==0:
            break
            
    x_size = coors_img[1] - coors_img[0] +1
    y_size = coors_img[3] - coors_img[2] +1
    
    if (x_size) !=64 :
        if x_size%2 == 0:
            coors_img[0]-=(64-x_size)/2
            coors_img[1]+=(64-x_size)/2
        elif x_size%2 != 0:
            coors_img[0]-=((64-x_size)/2) 
            coors_img[1]+=((64-x_size)/2) +1          
        
    if (y_size) !=64 :
        if y_size%2 == 0:
            coors_img[2]-=(64-y_size)/2
            coors_img[3]+=(64-y_size)/2
        elif y_size%2 !=0 :
            coors_img[2]-=((64-y_size)/2) 
            coors_img[3]+=((64-y_size)/2) +1
    
    cropI = image[int(coors_img[0]):int(coors_img[1]+1), int(coors_img[2]):int(coors_img[3]+1)]
    cropM = mask_img[int(coors_img[0]):int(coors_img[1]+1), int(coors_img[2]):int(coors_img[3]+1)]
    cv2.imwrite("./Augmentation/None/"+image_name+"_"+str(n)+".jpg", cropI)
    cv2.imwrite("./Augmentation/NoneMask/"+mask_name+"_"+str(n)+".jpg", cropM)

In [8]:
train_images=explore_dir("./dataset/train/images/",0)  # train images' name and locations
train_labels=explore_dir("./dataset/train/labels/",0)  # train masks' name and locations

In [9]:
for i in range(len(train_images[0])):
    img=cv2.imread(train_images[1][i])  # load images locations
    label_name=train_images[0][i][:-8]+"r"+train_images[0][i][-7:]
    mask=cv2.imread("./dataset/train/labels/"+label_name)  # load masks locations 

    # Crop Resion of Interest from Original images
    Crop_ROI(img,mask,train_images[0][i][:-4:],label_name)
    # Crop None ROI from Orininal images
    for k in range(8):
        Crop_None(img,mask,train_images[0][i][:-4:],label_name,k+1)